In [ ]:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import os
import sys
from statistics import mean
import pickle

%matplotlib inline
pd.set_option('display.max_colwidth', 200)


def handle_helper_functions():
    print(sys.path)
    directory_path = os.path.abspath(os.path.join('../../helper'))
    if directory_path not in sys.path:
        sys.path.append(directory_path)    

    translation_path=os.path.abspath(os.path.join('../../utils')) 
    if translation_path not in sys.path:
        sys.path.append(translation_path)   
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/test/helper')                                       
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/utils')
    sys.path.append('/Users/learn/Desktop/Projects/machine-translation/data')


    lstm_helper_path = os.path.abspath(os.path.join('./create-model/lstm/'))
    if lstm_helper_path not in sys.path:
        sys.path.append(lstm_helper_path)    

    print(sys.path)
    

handle_helper_functions()

### Importing Path of Helper Functions

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import tensorflow_text as text
from scoreTest import get_cosine_val, get_BLEU_score, get_ROUGE_score
from translate import  translate_sentence
from preprocessing_text import removePunctuation, toLowercase
from iit_dataset import createDataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [ ]:
data_size=15000

pool_oftexts, pairs =createDataset(data_size=data_size, type="train")
dataset= pool_oftexts

In [ ]:
# How many sentences will be used
# Limit the sentences to 10.000 on Kaggle to avoid exceding the
# available RAM space
# Build a generator to avoid this issue

total_sentences = 1000

# Load the dataset

# What proportion of the sentences will be used for the test set
test_proportion = 0.1
train_test_threshold = int( (1-test_proportion) * total_sentences)

printmd(f'## {total_sentences} "parallel sentences" will be loaded (original sentence + its translation)')
printmd(f'## {train_test_threshold} "parallel sentences" will be used to train the model')
printmd(f'## {total_sentences-train_test_threshold} "parallel sentences" will be used to test the model')

Source and Target Index

In [ ]:
idx_src=0
idx_tar = 1

In [ ]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, random_state=0)
dataset.iloc[1000:1010]

In [ ]:
def clean(string):
    # Clean the string
    string = string.replace("\u202f"," ") # Replace no-break space with space
    string = string.lower()
    
    # Delete the punctuation and the numbers
    for p in punctuation + "«»" + "0123456789":
        string = string.replace(p," ")
        
    string = re.sub('\s+',' ', string)
    string = string.strip()
           
    return string

# Clean the sentences
dataset["eng"] = dataset["eng"].apply(lambda x: clean(x))
dataset["hindi"] = dataset["hindi"].apply(lambda x: clean(x))

# Select one part of the dataset
dataset = dataset.values
dataset = dataset[:total_sentences]

# split into train/test
train, test = dataset[:train_test_threshold], dataset[train_test_threshold:]

# Define the name of the source and of the target
# This will be used in the outputs of this notebook
source_str, target_str = "Hindi", "English"

# The index in the numpy array of the source and of the target


In [ ]:
pd.DataFrame(dataset[1000:1010])


In [ ]:
def create_tokenizer(lines):
    # fit a tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 

def encode_sequences(tokenizer, length, lines):
    # encode and pad sequences
    X = tokenizer.texts_to_sequences(lines) # integer encode sequences
    X = pad_sequences(X, maxlen=length, padding='post') # pad sequences with 0 values
    return X
 
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y
 
# Prepare target tokenizer
tar_tokenizer = create_tokenizer(dataset[:, idx_tar]) #save
tar_vocab_size = len(tar_tokenizer.word_index) + 1  #save
tar_length = 15  #save
printmd(f'\nTarget ({target_str}) Vocabulary Size: {tar_vocab_size}')
printmd(f'Target ({target_str}) Max Length: {tar_length}')

# Prepare source tokenizer
src_tokenizer = create_tokenizer(dataset[:, idx_src])  #save
src_vocab_size = len(src_tokenizer.word_index) + 1  #save
src_length = 15  #save
printmd(f'\nSource ({source_str}) Vocabulary Size: {src_vocab_size}')
printmd(f'Source ({source_str}) Max Length: {src_length}\n')
 
# Prepare training data
trainX = encode_sequences(src_tokenizer, src_length, train[:, idx_src])
trainY = encode_sequences(tar_tokenizer, tar_length, train[:, idx_tar])
trainY = encode_output(trainY, tar_vocab_size)

# Prepare test data
testX = encode_sequences(src_tokenizer, src_length, test[:, idx_src])
testY = encode_sequences(tar_tokenizer, tar_length, test[:, idx_tar])
testY = encode_output(testY, tar_vocab_size)

In [ ]:
def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Create the model
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model
 
# Create model
model = create_model(src_vocab_size, tar_vocab_size, src_length, tar_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

history = model.fit(trainX, 
          trainY, 
          epochs=200,#200 
          batch_size=64, 
          validation_split=0.1, 
          verbose=1,
          callbacks=[
                        EarlyStopping(
                        monitor='val_loss',
                        patience=10,
                        restore_best_weights=True
                    )
            ])


model.summary()
# save model in computer '/Users/learn/Desktop/Projects/machine-translation/data'
# model.save()
pd.DataFrame(history.history).plot()
plt.title("Loss")
plt.show()

### Load Previously Saved Model

In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('lstm_model')
# model.summary()

In [ ]:
def word_for_id(integer, tokenizer):
    # map an integer to a word
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
def predict_seq(model, tokenizer, source):
    # generate target from a source sequence
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)
    
def create_dataframe_to_score(model, tokenizer, sources, raw_dataset):
    # Get the bleu score of a model
    actual, predicted, actual_rouge , cosine_value_list= [], [],[], []
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_seq(model, tar_tokenizer, source)
        raw_src, raw_target = raw_dataset[i]
        actual.append([raw_target.split()])
        actual_rouge.append(raw_target.split())
        predicted.append(translation.split())
        #######################################################################
                    ####   Calculate Cosine Value   ####

        cosine_value= get_cosine_val (translation,raw_target )
        cosine_value_list.append(cosine_value)
        #######################################################################
        # print("predicted ",translation.split())    
   
    average_cosine= mean(cosine_value_list)
    return actual, predicted, actual_rouge, average_cosine

actual, predicted, actual_rouge, average_cosine =create_dataframe_to_score(model, tar_tokenizer, testX, test)

In [ ]:
def bleu_score(actual, predicted):
    # Get the bleu score of a model
    bleu_dic = {}
    bleu_dic['bleu-1-grams'] = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
    bleu_dic['bleu-1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
    bleu_dic['bleu-1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
    bleu_dic['bleu-1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
    
    return bleu_dic

def calculate_ROUGE(actual, predicted):
    hypotheses = tf.ragged.constant(predicted)
    references = tf.ragged.constant(actual)

    rouge_test= text.metrics.rouge_l(hypotheses, references)
    f_measure_list= rouge_test.f_measure
    p_measure_list= rouge_test.p_measure
    r_measure_list= rouge_test.r_measure
  

    f_measure_average= (f_measure_list.numpy()).mean()
    p_measure_average= (p_measure_list.numpy()).mean()
    r_measure_average=(r_measure_list.numpy()).mean()
    return { "f_measure_average":f_measure_average, "p_measure_average":p_measure_average, "r_measure_average":r_measure_average  }  




In [ ]:
# result= get_cosine_val (actual[0], predicted[0])

# result= get_cosine_val ("I love cow","i love grass" )
# result

actual[0]

In [ ]:
predicted[0]

In [ ]:
# Compute the BLEU Score
# bleu_train = bleu_score(model, tar_tokenizer, trainX, train)
bleu_test = bleu_score(actual, predicted)
rouge_test = calculate_ROUGE(actual=actual_rouge, predicted=predicted)

In [ ]:
print(rouge_test)
print(bleu_test)
print(average_cosine)

In [ ]:
# plt.bar(x = bleu_train.keys(), height = bleu_train.values())
# plt.title("BLEU Score with the training set")
# plt.ylim((0,1))
# plt.show()

In [ ]:
plt.bar(x = bleu_test.keys(), height = bleu_test.values())
plt.title("BLEU Score with the test set")
plt.ylim((0,1))
plt.show()

In [ ]:
plt.bar(x = rouge_test.keys(), height = rouge_test.values())
plt.title("ROUGE Score with the test set")
plt.ylim((0,1))
plt.show()

In [ ]:
plt.bar(x = "cosine_average", height = average_cosine)
plt.title("Cosine Score with the test set")
plt.ylim((0,1))
plt.show()

In [ ]:
keys=["Dataset Size", *rouge_test.keys(), "cosine_similarity", *bleu_test.keys()]

values=[total_sentences,  *rouge_test.values(), average_cosine, *bleu_test.values() ]

table =pd.DataFrame(columns=keys, data=[values])
table


In [ ]:
import matplotlib.pyplot as plot
from matplotlib.ticker import ScalarFormatter
#,figsize=(10,15)

table.plot.bar(x="Dataset Size",figsize=(15,10))
plot.show(block=True)


###  Save Model and Parameters

In [52]:
import json
import time

def saveAnalytics(analytics_table):
    import os
    import json
    import time
    fname = "../../analytics/lstm/"+"analytics.json"

    if os.path.exists(fname):
        # read existing file and append new data
        with open(fname,"r") as f:
            df = pd.read_json(fname)
            new_data=(analytics_table.to_numpy())[0] 
            df.loc[len(df.index)] = new_data # add another row
            print("Update ",df.to_numpy() )
            df.to_json(r'../../analytics/lstm/'+"analytics.json",orient='records')
    else:
        # create new json
        print("Create ",analytics_table )
        analytics_table.to_json(r'../../analytics/lstm/'+"analytics.json",orient='records')


def save_models_and_parameters(total_sentences,model,src_tokenizer, tar_tokenizer, src_length, tar_length, src_vocab_size,tar_vocab_size, analytics_table ):
    model_name = str(total_sentences)
    path="../../model/lstm/"+model_name+"/"
    
    src_parameters={
        'src_length': src_length,
        'src_vocab_size': src_vocab_size,
    }
    src_tokenizer= src_tokenizer

    target_parameters={
        'target_length': tar_length,
        'target_vocab_size': tar_vocab_size,
    }
    target_tokenizer= tar_tokenizer

    model.save(path+'lstm_model' ) 
    with open(path+'src_parameters.pickle', 'wb') as handle:
        pickle.dump(src_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(path+'src_tokenizer.pickle', 'wb') as handle:
        pickle.dump(src_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(path+'target_parameters.pickle', 'wb') as handle:
        pickle.dump(target_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(path+'target_tokenizer.pickle', 'wb') as handle:
        pickle.dump(target_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    saveAnalytics(analytics_table=analytics_table)
   

   
save_models_and_parameters(total_sentences,model,src_tokenizer, tar_tokenizer, src_length, tar_length, src_vocab_size,tar_vocab_size, table )



INFO:tensorflow:Assets written to: ../../model/lstm/1000/lstm_model/assets


INFO:tensorflow:Assets written to: ../../model/lstm/1000/lstm_model/assets


Update  [[1.00000000e+004 7.64712036e-001 7.75633931e-001 7.64969170e-001
  7.86634681e-001 7.47674297e-001 6.53796439e-001 5.93642770e-001
  4.87186522e-001]
 [1.00000000e+003 3.24609280e-002 9.00000036e-002 2.00046729e-002
  5.06093138e-002 1.61748980e-003 6.93011957e-157 1.81453755e-187
  1.43446621e-233]
 [1.00000000e+003 3.24609280e-002 9.00000036e-002 2.00046729e-002
  5.06093138e-002 1.61748975e-003 6.93011957e-157 1.81453755e-187
  1.43446621e-233]]
